In [1]:
import os

In [2]:
os.chdir('../')

In [3]:
pwd

'/home/cloudcraftz/End-To-End-Wine-Quality'

In [23]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig():
    root_dir: Path
    unzip_data_dir: Path
    STATUS_FILE: str
    all_schema: dict

In [24]:
from WineQuality.constants import *
from WineQuality.utils.common import read_yaml, create_directories

In [25]:
class ConfigurationManager:
    
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH,schema_filepath = SCHEMA_FILE_PATH):
        
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = config.root_dir,
            unzip_data_dir = config.unzip_data_dir,
            STATUS_FILE = config.STATUS_FILE,
            all_schema = schema
        )
       
        return data_validation_config

In [26]:
import os
import pandas as pd
import urllib.request as request
import zipfile
from WineQuality import logger
from WineQuality.utils.common import get_size

In [27]:
class DataValidation:

    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:

        try:
            validation_status = True

            data = pd.read_csv(self.config.unzip_data_dir)

            column_info = dict(data.dtypes)

            all_schema = self.config.all_schema

            for col, dtype in column_info.items():
                expected_dtype = all_schema.get(col, None)

                if expected_dtype is None:
                    logger.error(f"Column {col} not in schema")
                    validation_status = False
                    break

                else:
                    if str(dtype) != str(expected_dtype):
                        logger.error(f"Column {col} dtype mismatch")
                        validation_status = False
                        break
            
            # Write final validation status to file
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation Status: {validation_status}")
                
            return validation_status
        
        except Exception as e:
            raise e


                    
                

        
        

In [30]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-03-16 16:16:09,874: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-03-16 16:16:09,875: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-16 16:16:09,877: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-16 16:16:09,877: INFO: common: created directory at: artifacts]
[2025-03-16 16:16:09,878: INFO: common: created directory at: artifacts/data_validation]


In [9]:
data = pd.read_csv('/home/cloudcraftz/End-To-End-Wine-Quality/artifacts/data_ingestion/winequality-red.csv')

In [11]:
list(data.columns)

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol',
 'quality']

In [16]:
d = dict(data.dtypes)

In [20]:
for column, data_type in d.items():
    print(str(data_type))

float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
float64
int64
